<a href="https://colab.research.google.com/github/akshaya-nagarajan/DeepLearningProjects/blob/master/Assignment_5/Assignment5_NeuralMachineTranslationLSTMGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'DeepLearningProjects/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
filePath = root_dir+'deu.txt'

In [0]:
with open(filePath, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [0]:
rows = len(lines)

In [0]:
trainlen = 100000

In [0]:
trainlen

100000

In [0]:
itexts = []
ttexts = []
for i in range(trainlen):
  text = lines[i]
  itext, ttext, _ = text.split('\t')

  ttext = '\t' + ttext + '\n'
  itexts.append(itext)
  ttexts.append(ttext)

In [0]:
engchars = set()
gerchars = set()
max_inp_len = 0
max_out_len = 0
for itext in itexts:
  max_inp_len = max(len(itext), max_inp_len)
  for char in itext:
    if char not in engchars:
      engchars.add(char)
for ttext in ttexts:
  max_out_len = max(len(ttext), max_out_len)
  for char in ttext:
    if char not in gerchars:
      gerchars.add(char)

In [0]:
engchars = sorted(list(engchars))
gerchars = sorted(list(gerchars))
inp_tokens = len(engchars)
out_tokens = len(gerchars)

In [0]:
inp_token_idx = {}
for i, c in enumerate(engchars):
  inp_token_idx[c] = i

target_token_idx = {}
for i, c in enumerate(gerchars):
  target_token_idx[c] = i

In [0]:
encoder_input_data = np.zeros(
    (len(itexts), max_inp_len, inp_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(itexts), max_out_len, out_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(itexts), max_out_len, out_tokens),
    dtype='float32')

In [0]:
for i, (itext, ttext) in enumerate(zip(itexts, ttexts)):
    for t, char in enumerate(itext):
        encoder_input_data[i, t, inp_token_idx[char]] = 1.
    encoder_input_data[i, t + 1:, inp_token_idx[' ']] = 1.
    for t, char in enumerate(ttext):
        decoder_input_data[i, t, target_token_idx[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_idx[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_idx[' ']] = 1.
    decoder_target_data[i, t:, target_token_idx[' ']] = 1.

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [0]:
batch_size = 100
epochs = 10
latent_dim = 256

In [0]:
encoder_inputs = Input(shape=(None, inp_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [0]:
decoder_inputs = Input(shape=(None, out_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(out_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('/content/gdrive/My Drive/258-DeepLearning/Data/language/translate_2.h5')

Train on 80000 samples, validate on 20000 samples
Epoch 1/30
80000/80000 [==============================] - 418s 5ms/step - loss: 0.4507 - accuracy: 0.8769 - val_loss: 0.4408 - val_accuracy: 0.8694
Epoch 2/30
80000/80000 [==============================] - 415s 5ms/step - loss: 0.2826 - accuracy: 0.9159 - val_loss: 0.3567 - val_accuracy: 0.8935
Epoch 3/30
80000/80000 [==============================] - 411s 5ms/step - loss: 0.2397 - accuracy: 0.9285 - val_loss: 0.3224 - val_accuracy: 0.9033
Epoch 4/30
80000/80000 [==============================] - 404s 5ms/step - loss: 0.2175 - accuracy: 0.9350 - val_loss: 0.3023 - val_accuracy: 0.9094
Epoch 5/30
80000/80000 [==============================] - 400s 5ms/step - loss: 0.2030 - accuracy: 0.9395 - val_loss: 0.2894 - val_accuracy: 0.9137
Epoch 6/30
80000/80000 [==============================] - 393s 5ms/step - loss: 0.1924 - accuracy: 0.9426 - val_loss: 0.2817 - val_accuracy: 0.9160
Epoch 7/30
80000/80000 [==============================] - 391s

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [0]:
inp_inv_idx = {}
target_inv_idx = {}
for char, i in inp_token_idx.items():
  inp_inv_idx[i] = char

for char, i in target_token_idx.items():
  target_inv_idx[i] = char

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, out_tokens))
    target_seq[0, 0, target_token_idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_inv_idx[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_out_len):
            stop_condition = True

        target_seq = np.zeros((1, 1, out_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [0]:
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', itexts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Geh weg!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Run.
Decoded sentence: Lacht das!

-
Input sentence: Wow!
Decoded sentence: Frage get!

-
Input sentence: Wow!
Decoded sentence: Frage get!

-
Input sentence: Fire!
Decoded sentence: Schneide!

-
Input sentence: Help!
Decoded sentence: Zuschalle ist es.

-
Input sentence: Help!
Decoded sentence: Zuschalle ist es.

